# HyDE (Hypothetical Document Embeddings) Demo

HyDE improves retrieval by using an LLM to generate hypothetical documents that would answer a user's question. Rather than directly retrieving documents based on the original query, HyDE first generates a relevant passage that could answer the question, then uses that generated passage for retrieval. This approach better aligns the query semantics with the document space and can improve retrieval quality.

## What this notebook contains
- Using an LLM to generate hypothetical documents that answer a given question.
- Creating embeddings from the generated documents and storing them in a vector database.
- Retrieving relevant documents using the generated hypothetical documents.
- Building a RAG pipeline that answers the question using the retrieved context.

## References
https://arxiv.org/abs/2212.10496

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter  
from langchain_community.document_loaders import WebBaseLoader  
from langchain_community.vectorstores import Chroma  
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser  
from langchain_openai import ChatOpenAI, OpenAIEmbeddings 
from langchain.prompts import ChatPromptTemplate
from langchain.load import dumps, loads
from langchain.schema import Document
import yaml
import bs4  
import os

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# Get the current working directory
cwd = os.getcwd()

# Build the path to config.yaml
config_path = os.path.join(cwd, '..', 'configs', 'config.yaml')

# Normalize the path
config_path = os.path.abspath(config_path)

# Load credential from config file
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# Set environment variables
os.environ['LANGCHAIN_API_KEY'] = config['API']['LANGCHAIN']
os.environ['OPENAI_API_KEY'] = config['API']['OPENAI']

# Configure chat LLM (deterministic)
llm = ChatOpenAI(temperature=0) 

In [3]:
# Create a loader that fetches and parses the target web page
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),  # tuple of URLs to load
    bs_kwargs=dict(  # pass BeautifulSoup-specific kwargs to limit parsing
        parse_only=bs4.SoupStrainer(  # only parse these parts of the page to reduce noise
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

# Fetch and return a list of Document objects
docs = loader.load()  

# Split long documents into smaller overlapping chunks suitable for embeddings
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splits = text_splitter.split_documents(docs)  # list of smaller document chunks

# Create embeddings and store them in a vector DB (Chroma)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())  # uses OpenAI embeddings under the hood

# Create a retriever to fetch relevant docs (return the top 1 results)
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [4]:
# HyDE document generation
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

# Build the HyDE chain
generate_docs_for_retrieval = (
    prompt_hyde | llm | StrOutputParser() 
)

# Run
question = "What is task decomposition for LLM agents?"
generated_docs = generate_docs_for_retrieval.invoke({"question":question})

In [5]:
# Wrap strings into Document objects
docs = [Document(page_content=text) for text in generated_docs]

# Split long documents into smaller overlapping chunks suitable for embeddings
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splits = text_splitter.split_documents(docs)  # list of smaller document chunks

# Create embeddings and store them in a vector DB (Chroma)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())  # uses OpenAI embeddings under the hood

# Create a retriever to fetch relevant docs (return the top 1 results)
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever 
retrieved_docs = retrieval_chain.invoke({"question":question})
retrieved_docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.')]

In [6]:
# RAG - build and run final chain that answers the question using retrieved context
template = """
Answer the following question based on this context:

{context}

Question: {question}
"""

# Build the prompt object
prompt = ChatPromptTemplate.from_template(template)

# Compose pipeline: retrieval_chain -> prompt -> llm -> parse
final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()  # parse final output to string
)

# Execute the chain and get the answer
final_rag_chain.invoke({"context":retrieved_docs,"question":question})

'Task decomposition for LLM agents involves breaking down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.'